In [1]:
import requests
import pandas as pd
import pickle
from statistics import mean
import os
import json
import yfinance as yf

In [2]:
def get_all_ratios_data (local_address):
    ratios_data={}
    files=os.listdir(local_address)
    
    for file in files:
        if file[-5:] == '.json' :
            with open(local_address+file) as jsonFile:
                ratios_data[file[:-5]]=json.load(jsonFile)
                jsonFile.close()
    
    return ratios_data

In [3]:
def get_ratios (ticker):
    financials = requests.request("GET", urls['financials'], headers=headers, params={"symbol":ticker})
    analysis = requests.request("GET", urls['analysis'], headers=headers, params={"symbol":ticker})
    statistics = requests.request("GET", urls['statistics'], headers=headers, params={"symbol":ticker})

    totalCurrentAssets=financials.json()['balanceSheetHistory']['balanceSheetStatements'][-1]['totalCurrentAssets']['raw']
    totalCurrentLiabilities=financials.json()['balanceSheetHistory']['balanceSheetStatements'][-1]['totalCurrentLiabilities']['raw']
    cashRatio=totalCurrentAssets/totalCurrentLiabilities

    ratios=[
            analysis.json()['financialData']['currentRatio']['raw'],
            analysis.json()['financialData']['quickRatio']['raw'],
            cashRatio,
            analysis.json()['financialData']['debtToEquity']['raw']/100,
            analysis.json()['financialData']['returnOnEquity']['raw'], 
            analysis.json()['financialData']['profitMargins']['raw'],
            analysis.json()['summaryDetail']['trailingPE']['raw'], 
            analysis.json()['summaryDetail']['priceToSalesTrailing12Months']['raw'],
            statistics.json()['defaultKeyStatistics']['priceToBook']['raw'],
            financials.json()['price']['regularMarketPrice']['raw']
         ]
    return ratios

In [7]:
tickers=['WMT','MSFT','GOOGL']

urls = { "financials":"https://yh-finance.p.rapidapi.com/stock/v2/get-financials",
         "analysis" : "https://yh-finance.p.rapidapi.com/stock/v2/get-analysis",
        "statistics" : "https://yh-finance.p.rapidapi.com/stock/v3/get-statistics"
       }

headers = {
    'x-rapidapi-host': "yh-finance.p.rapidapi.com",
    'x-rapidapi-key': "9824c59e6emsh42f5c3812ff0e1bp147eadjsn0a78eba1599f"
    }

ratios_dic={}
print("--->STARTING<---")
for element in tickers:
    try :
        print('Trying with: '+element,end=" ")
        ratios_dic[element]=get_ratios(element)
        print ("Success")
    except :
        print('Failed')
print("--->FINISHED<---")

--->STARTING<---
Trying with: WMT Success
Trying with: MSFT Success
Trying with: GOOGL Success
--->FINISHED<---


In [11]:
data_address='./Data/fundamental_analysis/ticker_ratios/'
all_tickers_ratios=get_all_ratios_data(data_address)


In [6]:
ratios=['Current Ratio','Quick Ratio','Debt\\/Equity','Inventory Turnover','Days Inventory','Asset Turnover','Return on Equity %','Net Margin %']
pd.DataFrame(all_tickers_ratios).loc[ratios]


KeyError: "None of [Index(['Current Ratio', 'Quick Ratio', 'Debt\\/Equity', 'Inventory Turnover',\n       'Days Inventory', 'Asset Turnover', 'Return on Equity %',\n       'Net Margin %'],\n      dtype='object')] are in the [index]"

In [6]:
ratiosNames=['currentRatio','quickRatio','cashRatio','debtToEquity','returnOnEquity','profitMargins','trailingPE',
       'priceToSalesTrailing12Months','priceToBook','regularMarketPrice']

valid_tickersNames=output.keys()

stocks_df=pd.DataFrame(output.values(),columns=ratiosNames,index=valid_tickersNames).transpose()
stocks_df['MarketAverage']=stocks_df.mean(axis='columns')
stocks_df

,MarketAverage
currentRatio,NaN
quickRatio,NaN
cashRatio,NaN
debtToEquity,NaN
returnOnEquity,NaN
profitMargins,NaN
trailingPE,NaN
priceToSalesTrailing12Months,NaN
priceToBook,NaN
regularMarketPrice,NaN


In [7]:
evaluation_df=stocks_df.loc[['trailingPE','priceToSalesTrailing12Months','priceToBook','regularMarketPrice']]
evaluation_df

,MarketAverage
trailingPE,NaN
priceToSalesTrailing12Months,NaN
priceToBook,NaN
regularMarketPrice,NaN


In [8]:
results={}
for ticker in valid_tickersNames:
    value_per_ratio=evaluation_df[ticker]['regularMarketPrice']*evaluation_df['MarketAverage']/evaluation_df[ticker]
    results[ticker]={'MarketPrice':evaluation_df[ticker]['regularMarketPrice'],
                     'ExpectedValue':mean(list(value_per_ratio[:-1]))
                    }
pd.DataFrame(results.values(),index=valid_tickersNames)

""
